In [75]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support import expected_conditions as EC

service = Service(executable_path='./chromedriver')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

# Function to get all movie genres
def get_movie_genres():

    #assigning the URL with variable name url
    url = 'https://www.imdb.com/feature/genre/'
    #request allow you to send HTTP request
    response = requests.get(url,headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Specify the target title
    target_title = "Popular movies by genre"# Specify the target title


    # Find the div with the specified title
    target_div = soup.find('span',id='movie')


    # Check if the target div is found
    if target_div:
        # Find the parent section element
        section_element = target_div.find_parent('section', class_='ipc-page-section')

        # Check if the section element is found
        if section_element:
            # Find all the anchor tags within the section
            genre_anchors = section_element.find_all('a', class_='ipc-chip ipc-chip--on-base-accent2')

            # Extract the text content of each span element within the anchor tags
            genres = [anchor.find('span').text.strip() for anchor in genre_anchors]

            return genres
        else:
            return []
    else:
        return []


# Function to extract movie details from a given genre
def extract_movie_details(genre):
    url = f'https://m.imdb.com/chart/top/?genres={genre}&sort=rank%2Casc'
    driver.get(url)
    button = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, "list-view-option-detailed"))
        )

    # Click the button to get full information of movie
    button.click()

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract only the top 20 movie details
    ul_element = soup.find('ul', class_='ipc-metadata-list')

    # Select the top 20 <li> elements from the <ul>
    top_20_li_elements = ul_element.find_all('li', class_='ipc-metadata-list-summary-item')[:20]

    movies_data=[]

    # Loop through the selected <li> elements and extract details
    for movie_element in top_20_li_elements:
        movie_details = {}
        #Add title
        title_element = movie_element.find('h3', class_='ipc-title__text')
        movie_details['Title'] = title_element.text.strip().split('. ')[1]

        # div contains year and duration
        metadata_div = movie_element.find('div', class_='sc-43986a27-7 dBkaPT dli-title-metadata')

        # Find the next two spans
        next_spans = metadata_div.find_all_next('span', class_='sc-43986a27-8 jHYIIK dli-title-metadata-item')[:2]
    
        #Add Year
        movie_details['Year'] = next_spans[0].text.strip()

        #Add Duration     
        movie_details['Duration'] = next_spans[1].text.strip()

        #Add rating
        rating_element = movie_element.find('span', class_='ipc-rating-star--voteCount')
        movie_details['Rating'] = rating_element.previous_sibling.strip()

        #Add cast names

        # Find the target span containing the specified string
        target_span = soup.find('span', class_='sc-53c98e73-7 hwuwjU', string='Stars')

        # Initialize an array to store the names
        names_array = []

        # Check if the target span is found
        if target_span:
            # Find the next three spans
            next_spans = target_span.find_all_next('span', class_='sc-53c98e73-6 fXEUnK')[:3]


            # Extract names from the next three spans
            for span in next_spans:
                actor_name_element = span.find('a', class_='ipc-link')
                if actor_name_element:
                    actor_name = actor_name_element.text.strip()
                    names_array.append(actor_name)

        movie_details['Casts'] = names_array

        #Add Director
        director_element = movie_element.find('span', class_='sc-53c98e73-6 fXEUnK')
        movie_details['Director'] = director_element.find_next('a', class_='ipc-link').text.strip()

        #Add movie_id
        link_element = movie_element.find('a', class_='ipc-title-link-wrapper')
        movie_details['Movie_id'] = link_element['href'].split('?')[0].split('/title/')[1].split('/')[0]

        #Add Genre
        movie_details['Genre'] = genre


        movies_data.append(movie_details)


    
    return movies_data


# Function to extract reviews for a given movie
def extract_movie_reviews(movie_id):

    #assigning the URL with variable name url
    url = f'https://m.imdb.com/title/{movie_id}/reviews?ref_=tt_urv'
    #request allow you to send HTTP request
    response = requests.get(url,headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')



    # Find the reviews container div
    reviews_container = soup.find('div', id='reviews-container')

    # Select all reviews within the container
    reviews = reviews_container.find_all('div', class_='imdb-user-review')

    # Save the top 10 reviews
    top_10_reviews = reviews[:10]

    reviews=[]

    # Loop through the selected reviews and extract details
    for review in top_10_reviews:

        reviews_data={}
        reviews_data['movie_id'] = movie_id
        #Add Review_id
        review_id = review['data-review-id']
        reviews_data['review_id'] = review_id

        #Add title_id
        title_id = review['data-title-id']
        reviews_data['title_id'] = title_id

        #Add reviewer name
        reviewer_name = review.find('a', class_='display-name-link').text.strip()
        reviews_data['reviewer_name'] = reviewer_name

        #Add review date
        review_date = review.find('span', class_='review-date').text.strip()
        reviews_data['review_date'] = review_date

        #Add review title
        review_title = review.find('a', class_='title').text.strip()
        reviews_data['review_title'] = review_title

        #Add review_ text
        review_text = review.find('div', class_='text').text.strip()
        reviews_data['review_text'] = review_text

        

        reviews.append(reviews_data)
    return reviews



In [76]:
import mysql.connector

# Function to get a MySQL connection
def get_mysql_connection():
    return mysql.connector.connect(
        # host="localhost",
        # user="madhav",
        # password="iitkcse22",
        # database="moviedata"
        host='localhost',
        database='movie3',
        user='cpgrams',
        password='rajnath',
         auth_plugin = 'mysql_native_password',
        
    )

# Function to create a movies table if not exists
def create_movies_table(connection):
    cursor = connection.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS movies (
            id INT AUTO_INCREMENT PRIMARY KEY,
            movie_id VARCHAR(255),
            title VARCHAR(255),
            year VARCHAR(10),
            duration VARCHAR(255),
            director VARCHAR(255),
            cast VARCHAR(1000),
            rating FLOAT,
            genre VARCHAR(255)
        )
    """)
    connection.commit()
    cursor.close()

# Function to create a reviews table if not exists
def create_reviews_table(connection):
    cursor = connection.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS reviews (
            id INT AUTO_INCREMENT PRIMARY KEY,
            movie_id VARCHAR(255),
            review_id VARCHAR(255),
            title_id VARCHAR(255),
            reviewer_name VARCHAR(255),
            review_date VARCHAR(20),
            review_title VARCHAR(255),
            review_text TEXT,
            helpful_count INT
            
        )
    """)
    connection.commit()
    cursor.close()

# Function to insert movie details into the movies table
def insert_movie_details(connection, movie):
    cursor = connection.cursor()
    cursor.execute("""
        INSERT INTO movies (movie_id,title,year,duration ,director,cast ,rating ,genre )
        VALUES (%s, %s, %s, %s, %s,%s, %s, %s)
    """, (movie['Movie_id'],movie['Title'], movie['Year'],movie['Duration'], movie['Director'], ', '.join(movie['Casts']), movie['Rating'],movie['Genre']))
    connection.commit()
    cursor.close()


# Function to insert reviews into the reviews table
def insert_movie_reviews(connection,review_data):
    cursor = connection.cursor()
    for reviews in review_data:
        cursor.execute("""
            INSERT INTO reviews (movie_id,review_id,title_id,reviewer_name, review_date,review_title,review_text)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (reviews['movie_id'],reviews['review_id'],reviews['title_id'],reviews['reviewer_name'],reviews['review_date'],reviews['review_title'],reviews['review_text']))
    connection.commit()
    cursor.close()

if __name__ == "__main__":
    # Establish MySQL connection and create tables
    mysql_connection = get_mysql_connection()
    create_movies_table(mysql_connection)
    create_reviews_table(mysql_connection)

    genres = get_movie_genres()


    for genre in genres:
        print(f"Extracting data for {genre} movies...")
        movies = extract_movie_details(genre)
        print(movies)
        for movie in movies:
            # Save movie details to MySQL
            insert_movie_details(mysql_connection, movie)

            # Extract reviews for the movie
            reviews = extract_movie_reviews(movie['Movie_id'])
            print(reviews)

            # Save reviews to MySQL

            insert_movie_reviews(mysql_connection,reviews)

    # Close MySQL connection
    mysql_connection.close()
